## Data for Figure 8

#### Based on Elise Olson's https://github.com/SalishSeaCast/analysis-elise-2/blob/master/notebooks/modelEqs/NewRateComparison.ipynb notebook

In [1]:
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt
from salishsea_tools import bio_tools as bt, places
import xarray as xr
import os
import glob
%matplotlib inline

In [2]:
nml=bt.load_nml_bio(resDir='/ocean/eolson/MEOPAR/NEMO-3.6-code/NEMOGCM/CONFIG/SalishSeaCast/EXP00/',
                 nmlname='nampisprod',bioRefName='namelist_smelt_cfg_HC201905equiv',bioCfgName='namelist_smelt_cfg_HC201905equiv')

In [3]:
nml

Namelist([('zz_rate_r_diat', 6.0495e-05),
          ('zz_rate_r_myri', 2.22e-05),
          ('zz_rate_r_nano', 2.109e-05),
          ('zz_rate_maxtemp_diat', 26.0),
          ('zz_rate_maxtemp_myri', 31.0),
          ('zz_rate_maxtemp_nano', 31.0),
          ('zz_rate_temprange_diat', 14.0),
          ('zz_rate_temprange_myri', 13.0),
          ('zz_rate_temprange_nano', 13.0),
          ('zz_rate_iopt_diat', 45.0),
          ('zz_rate_iopt_myri', 37.0),
          ('zz_rate_iopt_nano', 10.0),
          ('zz_rate_gamma_diat', 0.0),
          ('zz_rate_gamma_myri', 0.0),
          ('zz_rate_gamma_nano', 0.0),
          ('zz_rate_k_si_diat', 2.2),
          ('zz_rate_k_si_myri', 0.0),
          ('zz_rate_k_si_nano', 0.0),
          ('zz_rate_kapa_diat', 1.0),
          ('zz_rate_kapa_myri', 0.5),
          ('zz_rate_kapa_nano', 0.3),
          ('zz_rate_k_diat', 2.0),
          ('zz_rate_k_myri', 0.5),
          ('zz_rate_k_nano', 0.2),
          ('zz_rate_si_ratio_diat', 1.8),
          

In [4]:
def phyto_Tdep_Factor(TT, zz_rate_maxtemp, zz_rate_temprange):
    if hasattr(TT,'__len__'): # assume 1-d array or similar and return array
        return np.array([phyto_Tdep_Factor(el,zz_rate_maxtemp, zz_rate_temprange) for el in TT])
    else:
        return np.exp(0.07 * (TT - 20)) * min(max((zz_rate_maxtemp - TT), 0.0),zz_rate_temprange) / (zz_rate_temprange + 1e-10)

In [5]:
def calc_T_Factors(TT,nampisprod):
    Tdep_Diat=phyto_Tdep_Factor(TT,nampisprod['zz_rate_maxtemp_diat'],nampisprod['zz_rate_temprange_diat'])
    Tdep_Myri=phyto_Tdep_Factor(TT,nampisprod['zz_rate_maxtemp_myri'],nampisprod['zz_rate_temprange_myri'])
    Tdep_Nano=phyto_Tdep_Factor(TT,nampisprod['zz_rate_maxtemp_nano'],nampisprod['zz_rate_temprange_nano'])
    return Tdep_Diat, Tdep_Myri, Tdep_Nano

In [6]:
### Import Temperature Files

In [7]:
## Original WY


monthly_array_temp_orig_slice = np.zeros([14,12,50,50])
# Load monthly averages
mask = xr.open_dataset('/data/eolson/results/MEOPAR/NEMO-forcing-new/grid/mesh_mask201702.nc')
slc = {'y': slice(450,500), 'x': slice(250,300)}
e3t, tmask = [mask[var].isel(z=slice(None, 27),**slc).values for var in ('e3t_0', 'tmask')]
years, variables = range(2007, 2021), ['votemper']
# Temporary list dict
data = {}
# Permanent aggregate dict
aggregates = {var: {} for var in variables}
monthlydat = {var: {} for var in variables}

### 2008 using higher temperature threshold       
# Add experiment year
for year in [2019]:
    # Initialize lists
    for var in variables: data[var] = []
    # Load monthly averages
    for month in range(1, 13):
        datestr = f'{year}{month:02d}'
        prefix = f'/data/sallen/results/MEOPAR/v201905r/SalishSea_1m_{datestr}_{datestr}'
        
        with xr.open_dataset(prefix + '_grid_T.nc') as ds:
            q = ds.votemper.isel(deptht=0, **slc).values
            q2 = q[0,:,:]
            monthly_array_temp_orig_slice[year-2007,month-1,:,:] = q2 #year2007 is index 0 along 1st dimension
            for var in ['votemper']:
                data[var].append(ds.votemper.isel(deptht=0, **slc).values)
    # Concatenate months
    for var in variables: aggregates[var][year] = np.concatenate(data[var]).mean(axis=0)


In [8]:
monthly_array_temp_orig_slice[monthly_array_temp_orig_slice == 0 ] = np.nan
monthly_array_temp_orig_slicemean = \
np.nanmean(np.nanmean(monthly_array_temp_orig_slice, axis = 2),axis = 2)
print(np.shape(monthly_array_temp_orig_slicemean))

(14, 12)


/tmp/ipykernel_3795255/1734163470.py:3: RuntimeWarning: Mean of empty slice
  np.nanmean(np.nanmean(monthly_array_temp_orig_slice, axis = 2),axis = 2)


In [9]:
## Experiment 9


monthly_array_temp_exp_slice = np.zeros([14,12,50,50])
# Load monthly averages
mask = xr.open_dataset('/data/eolson/results/MEOPAR/NEMO-forcing-new/grid/mesh_mask201702.nc')
slc = {'y': slice(450,500), 'x': slice(250,300)}
e3t, tmask = [mask[var].isel(z=slice(None, 27),**slc).values for var in ('e3t_0', 'tmask')]
years, variables = range(2007, 2021), ['votemper']
# Temporary list dict
data = {}
# Permanent aggregate dict
aggregates = {var: {} for var in variables}
monthlydat = {var: {} for var in variables}

for year in [2019]:
    # Initialize lists
    for var in variables: data[var] = []
    # Load monthly averages
    for month in range(1, 7):
        datestr = f'{year}{month:02d}'
        prefix = f'/data/sallen/results/MEOPAR/Karyn/01jan19_river08/SalishSea_1m_{datestr}_{datestr}'
        
        with xr.open_dataset(prefix + '_grid_T.nc') as ds:
            q = ds.votemper.isel(deptht=0, **slc).values
            q2 = q[0,:,:]
            monthly_array_temp_exp_slice[year-2007,month-1,:,:] = q2 #year2007 is index 0 along 1st dimension
            for var in ['votemper']:
                data[var].append(ds.votemper.isel(deptht=0, **slc).values)
    # Concatenate months
    for var in variables: aggregates[var][year] = np.concatenate(data[var]).mean(axis=0)

for year in [2019]:
    # Initialize lists
    for var in variables: data[var] = []
    # Load monthly averages
    for month in range(7, 10):
        datestr = f'{year}{month:02d}'
        prefix = f'/data/sallen/results/MEOPAR/Karyn/01jul19_river08/SalishSea_1m_{datestr}_{datestr}'
        
        with xr.open_dataset(prefix + '_grid_T.nc') as ds:
            q = ds.votemper.isel(deptht=0, **slc).values
            q2 = q[0,:,:]
            monthly_array_temp_exp_slice[year-2007,month-1,:,:] = q2 #year2007 is index 0 along 1st dimension
            for var in ['votemper']:
                data[var].append(ds.votemper.isel(deptht=0, **slc).values)
    # Concatenate months
    for var in variables: aggregates[var][year] = np.concatenate(data[var]).mean(axis=0)
        

for year in [2019]:
    # Initialize lists
    for var in variables: data[var] = []
    # Load monthly averages
    for month in range(10, 13):
        datestr = f'{year}{month:02d}'
        prefix = f'/data/sallen/results/MEOPAR/Karyn/11oct19_river08/SalishSea_1m_{datestr}_{datestr}'
        
        with xr.open_dataset(prefix + '_grid_T.nc') as ds:
            q = ds.votemper.isel(deptht=0, **slc).values
            q2 = q[0,:,:]
            monthly_array_temp_exp_slice[year-2007,month-1,:,:] = q2 #year2007 is index 0 along 1st dimension
            for var in ['votemper']:
                data[var].append(ds.votemper.isel(deptht=0, **slc).values)
    # Concatenate months
    for var in variables: aggregates[var][year] = np.concatenate(data[var]).mean(axis=0)


In [10]:
monthly_array_temp_exp_slice[monthly_array_temp_exp_slice == 0 ] = np.nan
monthly_array_temp_exp_slicemean = \
np.nanmean(np.nanmean(monthly_array_temp_exp_slice, axis = 2),axis = 2)
print(np.shape(monthly_array_temp_exp_slicemean))

(14, 12)


/tmp/ipykernel_3795255/2338418733.py:3: RuntimeWarning: Mean of empty slice
  np.nanmean(np.nanmean(monthly_array_temp_exp_slice, axis = 2),axis = 2)


In [11]:
monthly_array_temp_exp_slicemean[12,:]

array([ 6.68492084,  5.18552462,  7.25557932, 10.16054886, 14.0224531 ,
       17.57203394, 20.68068024, 20.043708  , 16.67098008, 11.61331057,
        8.74718019,  6.62090044])

In [12]:
monthly_array_temp_orig_slicemean[12,:]

array([ 6.66574269,  5.18524643,  7.24007046, 10.16174981, 14.03568474,
       17.7114818 , 20.61275981, 20.03991294, 16.73050814, 11.58581398,
        8.60023705,  6.47922814])

### Temperature response factors


In [13]:
TdepDiatOrig,__,TdepNanoOrig=calc_T_Factors(monthly_array_temp_orig_slicemean[12,:],nml)
TdepDiatExp,__,TdepNanoExp=calc_T_Factors(monthly_array_temp_exp_slicemean[12,:],nml)

#### Open Nitrate files

In [14]:
## Original WY

monthly_array_nitrate_orig_slice = np.zeros([14,12,50,50])
# Load monthly averages
mask = xr.open_dataset('/data/eolson/results/MEOPAR/NEMO-forcing-new/grid/mesh_mask201702.nc')
slc = {'y': slice(450,500), 'x': slice(250,300)} 
e3t, tmask = [mask[var].isel(z=slice(None, 27),**slc).values for var in ('e3t_0', 'tmask')]
years, variables = range(2007, 2021), ['nitrate']
# Temporary list dict
data = {}
# Permanent aggregate dict
aggregates = {var: {} for var in variables}
monthlydat = {var: {} for var in variables}
        
# Add experiment year
for year in [2019]:
    # Initialize lists
    for var in variables: data[var] = []
    # Load monthly averages
    for month in range(1, 13):
        datestr = f'{year}{month:02d}'
        prefix = f'/data/sallen/results/MEOPAR/v201905r/SalishSea_1m_{datestr}_{datestr}'
        
        
        # Load grazing variables
        with xr.open_dataset(prefix + '_ptrc_T.nc') as ds:
            q = ds.nitrate.isel(deptht=0, **slc).values
            q2 = q[0,:,:]
            monthly_array_nitrate_orig_slice[year-2007,month-1,:,:] = q2 #year2007 is index 0 along 1st dimension
            for var in ['nitrate']:
                data[var].append(ds.nitrate.isel(deptht=0, **slc).values)
                
    # Concatenate months
    for var in variables: aggregates[var][year] = np.concatenate(data[var]).mean(axis=0)        


In [15]:
monthly_array_nitrate_orig_slice[monthly_array_nitrate_orig_slice == 0 ] = np.nan
monthly_array_nitrate_orig_slicemean = \
np.nanmean(np.nanmean(monthly_array_nitrate_orig_slice, axis = 2),axis = 2)
print(np.shape(monthly_array_nitrate_orig_slicemean))

(14, 12)


/tmp/ipykernel_3795255/3312634990.py:3: RuntimeWarning: Mean of empty slice
  np.nanmean(np.nanmean(monthly_array_nitrate_orig_slice, axis = 2),axis = 2)


In [16]:
## Experiment 9

monthly_array_nitrate_exp_slice = np.zeros([14,12,50,50])
# Load monthly averages
mask = xr.open_dataset('/data/eolson/results/MEOPAR/NEMO-forcing-new/grid/mesh_mask201702.nc')
slc = {'y': slice(450,500), 'x': slice(250,300)} 
e3t, tmask = [mask[var].isel(z=slice(None, 27),**slc).values for var in ('e3t_0', 'tmask')]
years, variables = range(2007, 2021), ['nitrate']
# Temporary list dict
data = {}
# Permanent aggregate dict
aggregates = {var: {} for var in variables}
monthlydat = {var: {} for var in variables}
        
# Add experiment year
for year in [2019]:
    # Initialize lists
    for var in variables: data[var] = []
    # Load monthly averages
    for month in range(1, 7):
        datestr = f'{year}{month:02d}'
        prefix = f'/data/sallen/results/MEOPAR/Karyn/01jan19_river08/SalishSea_1m_{datestr}_{datestr}'
        
        
        # Load grazing variables
        with xr.open_dataset(prefix + '_ptrc_T.nc') as ds:
            q = ds.nitrate.isel(deptht=0, **slc).values
            q2 = q[0,:,:]
            monthly_array_nitrate_exp_slice[year-2007,month-1,:,:] = q2 #year2007 is index 0 along 1st dimension
            for var in ['nitrate']:
                data[var].append(ds.nitrate.isel(deptht=0, **slc).values)
                
    # Concatenate months
    for var in variables: aggregates[var][year] = np.concatenate(data[var]).mean(axis=0)   
        

# Add experiment year
for year in [2019]:
    # Initialize lists
    for var in variables: data[var] = []
    # Load monthly averages
    for month in range(7, 10):
        datestr = f'{year}{month:02d}'
        prefix = f'/data/sallen/results/MEOPAR/Karyn/01jul19_river08/SalishSea_1m_{datestr}_{datestr}'
        
        
        # Load grazing variables
        with xr.open_dataset(prefix + '_ptrc_T.nc') as ds:
            q = ds.nitrate.isel(deptht=0, **slc).values
            q2 = q[0,:,:]
            monthly_array_nitrate_exp_slice[year-2007,month-1,:,:] = q2 #year2007 is index 0 along 1st dimension
            for var in ['nitrate']:
                data[var].append(ds.nitrate.isel(deptht=0, **slc).values)
                
    # Concatenate months
    for var in variables: aggregates[var][year] = np.concatenate(data[var]).mean(axis=0) 

        


for year in [2019]:
    # Initialize lists
    for var in variables: data[var] = []
    # Load monthly averages
    for month in range(10, 13):
        datestr = f'{year}{month:02d}'
        prefix = f'/data/sallen/results/MEOPAR/Karyn/11oct19_river08/SalishSea_1m_{datestr}_{datestr}'
        
        
        # Load grazing variables
        with xr.open_dataset(prefix + '_ptrc_T.nc') as ds:
            q = ds.nitrate.isel(deptht=0, **slc).values
            q2 = q[0,:,:]
            monthly_array_nitrate_exp_slice[year-2007,month-1,:,:] = q2 #year2007 is index 0 along 1st dimension
            for var in ['nitrate']:
                data[var].append(ds.nitrate.isel(deptht=0, **slc).values)
                
    # Concatenate months
    for var in variables: aggregates[var][year] = np.concatenate(data[var]).mean(axis=0) 




In [17]:
monthly_array_nitrate_exp_slice[monthly_array_nitrate_exp_slice == 0 ] = np.nan
monthly_array_nitrate_exp_slicemean = \
np.nanmean(np.nanmean(monthly_array_nitrate_exp_slice, axis = 2),axis = 2)
print(np.shape(monthly_array_nitrate_exp_slicemean))

(14, 12)


/tmp/ipykernel_3795255/1010059431.py:3: RuntimeWarning: Mean of empty slice
  np.nanmean(np.nanmean(monthly_array_nitrate_exp_slice, axis = 2),axis = 2)


In [18]:
## Original WY


monthly_array_silicon_orig_slice = np.zeros([14,12,50,50])
# Load monthly averages
mask = xr.open_dataset('/data/eolson/results/MEOPAR/NEMO-forcing-new/grid/mesh_mask201702.nc')
slc = {'y': slice(450,500), 'x': slice(250,300)} 
e3t, tmask = [mask[var].isel(z=slice(None, 27),**slc).values for var in ('e3t_0', 'tmask')]
years, variables = range(2007, 2021), ['silicon']
# Temporary list dict
data = {}
# Permanent aggregate dict
aggregates = {var: {} for var in variables}
monthlydat = {var: {} for var in variables}

        
# Add experiment year
for year in [2019]:
    # Initialize lists
    for var in variables: data[var] = []
    # Load monthly averages
    for month in range(1, 13):
        datestr = f'{year}{month:02d}'
        prefix = f'/data/sallen/results/MEOPAR/v201905r/SalishSea_1m_{datestr}_{datestr}'
        
        
        # Load grazing variables
        with xr.open_dataset(prefix + '_ptrc_T.nc') as ds:
            q = ds.silicon.isel(deptht=0, **slc).values
            q2 = q[0,:,:]
            monthly_array_silicon_orig_slice[year-2007,month-1,:,:] = q2 #year2007 is index 0 along 1st dimension
            for var in ['silicon']:
                data[var].append(ds.silicon.isel(deptht=0, **slc).values)
    
    # Concatenate months
    for var in variables: aggregates[var][year] = np.concatenate(data[var]).mean(axis=0)        

       

In [19]:
monthly_array_silicon_orig_slice[monthly_array_silicon_orig_slice == 0 ] = np.nan
monthly_array_silicon_orig_slicemean = \
np.nanmean(np.nanmean(monthly_array_silicon_orig_slice, axis = 2),axis = 2)
print(np.shape(monthly_array_silicon_orig_slicemean))

(14, 12)


/tmp/ipykernel_3795255/241793216.py:3: RuntimeWarning: Mean of empty slice
  np.nanmean(np.nanmean(monthly_array_silicon_orig_slice, axis = 2),axis = 2)


In [20]:
## Experiment 9

monthly_array_silicon_exp_slice = np.zeros([14,12,50,50])
# Load monthly averages
mask = xr.open_dataset('/data/eolson/results/MEOPAR/NEMO-forcing-new/grid/mesh_mask201702.nc')
slc = {'y': slice(450,500), 'x': slice(250,300)} 
e3t, tmask = [mask[var].isel(z=slice(None, 27),**slc).values for var in ('e3t_0', 'tmask')]
years, variables = range(2007, 2021), ['silicon']
# Temporary list dict
data = {}
# Permanent aggregate dict
aggregates = {var: {} for var in variables}
monthlydat = {var: {} for var in variables}
        
# Add experiment year
for year in [2019]:
    # Initialize lists
    for var in variables: data[var] = []
    # Load monthly averages
    for month in range(1, 7):
        datestr = f'{year}{month:02d}'
        prefix = f'/data/sallen/results/MEOPAR/Karyn/01jan19_river08/SalishSea_1m_{datestr}_{datestr}'
        
        
        # Load grazing variables
        with xr.open_dataset(prefix + '_ptrc_T.nc') as ds:
            q = ds.silicon.isel(deptht=0, **slc).values
            q2 = q[0,:,:]
            monthly_array_silicon_exp_slice[year-2007,month-1,:,:] = q2 #year2007 is index 0 along 1st dimension
            for var in ['silicon']:
                data[var].append(ds.silicon.isel(deptht=0, **slc).values)
                
    # Concatenate months
    for var in variables: aggregates[var][year] = np.concatenate(data[var]).mean(axis=0)   
        

# Add experiment year
for year in [2019]:
    # Initialize lists
    for var in variables: data[var] = []
    # Load monthly averages
    for month in range(7, 10):
        datestr = f'{year}{month:02d}'
        prefix = f'/data/sallen/results/MEOPAR/Karyn/01jul19_river08/SalishSea_1m_{datestr}_{datestr}'
        
        
        # Load grazing variables
        with xr.open_dataset(prefix + '_ptrc_T.nc') as ds:
            q = ds.silicon.isel(deptht=0, **slc).values
            q2 = q[0,:,:]
            monthly_array_silicon_exp_slice[year-2007,month-1,:,:] = q2 #year2007 is index 0 along 1st dimension
            for var in ['silicon']:
                data[var].append(ds.silicon.isel(deptht=0, **slc).values)
                
    # Concatenate months
    for var in variables: aggregates[var][year] = np.concatenate(data[var]).mean(axis=0) 


for year in [2019]:
    # Initialize lists
    for var in variables: data[var] = []
    # Load monthly averages
    for month in range(10, 13):
        datestr = f'{year}{month:02d}'
        prefix = f'/data/sallen/results/MEOPAR/Karyn/11oct19_river08/SalishSea_1m_{datestr}_{datestr}'
        
        
        # Load grazing variables
        with xr.open_dataset(prefix + '_ptrc_T.nc') as ds:
            q = ds.silicon.isel(deptht=0, **slc).values
            q2 = q[0,:,:]
            monthly_array_silicon_exp_slice[year-2007,month-1,:,:] = q2 #year2007 is index 0 along 1st dimension
            for var in ['silicon']:
                data[var].append(ds.silicon.isel(deptht=0, **slc).values)
                
    # Concatenate months
    for var in variables: aggregates[var][year] = np.concatenate(data[var]).mean(axis=0) 




In [21]:
monthly_array_silicon_exp_slice[monthly_array_silicon_exp_slice == 0 ] = np.nan
monthly_array_silicon_exp_slicemean = \
np.nanmean(np.nanmean(monthly_array_silicon_exp_slice, axis = 2),axis = 2)
print(np.shape(monthly_array_silicon_exp_slicemean))

(14, 12)


/tmp/ipykernel_3795255/4227105976.py:3: RuntimeWarning: Mean of empty slice
  np.nanmean(np.nanmean(monthly_array_silicon_exp_slice, axis = 2),axis = 2)


In [22]:

## Original WY

monthly_array_ammonium_orig_slice = np.zeros([14,12,50,50])
# Load monthly averages
mask = xr.open_dataset('/data/eolson/results/MEOPAR/NEMO-forcing-new/grid/mesh_mask201702.nc')
slc = {'y': slice(450,500), 'x': slice(250,300)} 
e3t, tmask = [mask[var].isel(z=slice(None, 27),**slc).values for var in ('e3t_0', 'tmask')]
years, variables = range(2007, 2021), ['ammonium']
# Temporary list dict
data = {}
# Permanent aggregate dict
aggregates = {var: {} for var in variables}
monthlydat = {var: {} for var in variables}

        
# Add experiment year
for year in [2019]:
    # Initialize lists
    for var in variables: data[var] = []
    # Load monthly averages
    for month in range(1, 13):
        datestr = f'{year}{month:02d}'
        prefix = f'/data/sallen/results/MEOPAR/v201905r/SalishSea_1m_{datestr}_{datestr}'
        
        
        # Load grazing variables
        with xr.open_dataset(prefix + '_ptrc_T.nc') as ds:
            q = ds.ammonium.isel(deptht=0, **slc).values
            q2 = q[0,:,:]
            monthly_array_ammonium_orig_slice[year-2007,month-1,:,:] = q2 #year2007 is index 0 along 1st dimension
            for var in ['ammonium']:
                data[var].append(ds.ammonium.isel(deptht=0, **slc).values)
    
    # Concatenate months
    for var in variables: aggregates[var][year] = np.concatenate(data[var]).mean(axis=0)        

       

In [23]:
monthly_array_ammonium_orig_slice[monthly_array_ammonium_orig_slice == 0 ] = np.nan
monthly_array_ammonium_orig_slicemean = \
np.nanmean(np.nanmean(monthly_array_ammonium_orig_slice, axis = 2),axis = 2)
print(np.shape(monthly_array_ammonium_orig_slicemean))

(14, 12)


/tmp/ipykernel_3795255/3976818894.py:3: RuntimeWarning: Mean of empty slice
  np.nanmean(np.nanmean(monthly_array_ammonium_orig_slice, axis = 2),axis = 2)


In [24]:
## Experiment 9

monthly_array_ammonium_exp_slice = np.zeros([14,12,50,50])
# Load monthly averages
mask = xr.open_dataset('/data/eolson/results/MEOPAR/NEMO-forcing-new/grid/mesh_mask201702.nc')
slc = {'y': slice(450,500), 'x': slice(250,300)} 
e3t, tmask = [mask[var].isel(z=slice(None, 27),**slc).values for var in ('e3t_0', 'tmask')]
years, variables = range(2007, 2021), ['ammonium']
# Temporary list dict
data = {}
# Permanent aggregate dict
aggregates = {var: {} for var in variables}
monthlydat = {var: {} for var in variables}
        
# Add experiment year
for year in [2019]:
    # Initialize lists
    for var in variables: data[var] = []
    # Load monthly averages
    for month in range(1, 7):
        datestr = f'{year}{month:02d}'
        prefix = f'/data/sallen/results/MEOPAR/Karyn/01jan19_river08/SalishSea_1m_{datestr}_{datestr}'
        
        
        # Load grazing variables
        with xr.open_dataset(prefix + '_ptrc_T.nc') as ds:
            q = ds.ammonium.isel(deptht=0, **slc).values
            q2 = q[0,:,:]
            monthly_array_ammonium_exp_slice[year-2007,month-1,:,:] = q2 #year2007 is index 0 along 1st dimension
            for var in ['ammonium']:
                data[var].append(ds.ammonium.isel(deptht=0, **slc).values)
                
    # Concatenate months
    for var in variables: aggregates[var][year] = np.concatenate(data[var]).mean(axis=0)   
        

# Add experiment year
for year in [2019]:
    # Initialize lists
    for var in variables: data[var] = []
    # Load monthly averages
    for month in range(7, 10):
        datestr = f'{year}{month:02d}'
        prefix = f'/data/sallen/results/MEOPAR/Karyn/01jul19_river08/SalishSea_1m_{datestr}_{datestr}'
        
        
        # Load grazing variables
        with xr.open_dataset(prefix + '_ptrc_T.nc') as ds:
            q = ds.ammonium.isel(deptht=0, **slc).values
            q2 = q[0,:,:]
            monthly_array_ammonium_exp_slice[year-2007,month-1,:,:] = q2 #year2007 is index 0 along 1st dimension
            for var in ['ammonium']:
                data[var].append(ds.ammonium.isel(deptht=0, **slc).values)
                
    # Concatenate months
    for var in variables: aggregates[var][year] = np.concatenate(data[var]).mean(axis=0) 

for year in [2019]:
    # Initialize lists
    for var in variables: data[var] = []
    # Load monthly averages
    for month in range(10, 13):
        datestr = f'{year}{month:02d}'
        prefix = f'/data/sallen/results/MEOPAR/Karyn/11oct19_river08/SalishSea_1m_{datestr}_{datestr}'
        
        
        # Load grazing variables
        with xr.open_dataset(prefix + '_ptrc_T.nc') as ds:
            q = ds.ammonium.isel(deptht=0, **slc).values
            q2 = q[0,:,:]
            monthly_array_ammonium_exp_slice[year-2007,month-1,:,:] = q2 #year2007 is index 0 along 1st dimension
            for var in ['ammonium']:
                data[var].append(ds.ammonium.isel(deptht=0, **slc).values)
                
    # Concatenate months
    for var in variables: aggregates[var][year] = np.concatenate(data[var]).mean(axis=0) 



In [25]:
monthly_array_ammonium_exp_slice[monthly_array_ammonium_exp_slice == 0 ] = np.nan
monthly_array_ammonium_exp_slicemean = \
np.nanmean(np.nanmean(monthly_array_ammonium_exp_slice, axis = 2),axis = 2)
print(np.shape(monthly_array_ammonium_exp_slicemean))

(14, 12)


/tmp/ipykernel_3795255/3603250393.py:3: RuntimeWarning: Mean of empty slice
  np.nanmean(np.nanmean(monthly_array_ammonium_exp_slice, axis = 2),axis = 2)


In [26]:
# for now just set light to constant and ignore 'limiter' and 'limval'
DiatLimOrig, __, NanoLimOrig = bt.calc_p_limiters(10*np.ones(np.shape(monthly_array_nitrate_orig_slicemean[12,:])),
                                               NO=monthly_array_nitrate_orig_slicemean[12,:],
                                               NH=monthly_array_ammonium_orig_slicemean[12,:],
                                               Si=monthly_array_silicon_orig_slicemean[12,:],
                                               tmask=np.ones(np.shape(monthly_array_nitrate_orig_slicemean[12,:])),
                                               nampisprod=nml)

In [27]:
# for now just set light to constant and ignore 'limiter' and 'limval'
DiatLimExp, __, NanoLimExp = bt.calc_p_limiters(10*np.ones(np.shape(monthly_array_nitrate_exp_slicemean[12,:])),
                                               NO=monthly_array_nitrate_exp_slicemean[12,:],
                                               NH=monthly_array_ammonium_exp_slicemean[12,:],
                                               Si=monthly_array_silicon_exp_slicemean[12,:],
                                               tmask=np.ones(np.shape(monthly_array_nitrate_exp_slicemean[12,:])),
                                               nampisprod=nml)

In [28]:
DiatLimOrig

{'ILim': array([0.51559692, 0.51559692, 0.51559692, 0.51559692, 0.51559692,
        0.51559692, 0.51559692, 0.51559692, 0.51559692, 0.51559692,
        0.51559692, 0.51559692]),
 'NLim': array([0.92177655, 0.91938623, 0.88324047, 0.71279276, 0.65085919,
        0.39827465, 0.38265472, 0.40354223, 0.77676333, 0.89482032,
        0.9016511 , 0.9135385 ]),
 'SiLim': array([0.95638142, 0.95736831, 0.94794749, 0.87408713, 0.83030771,
        0.91917568, 0.93623009, 0.93732263, 0.94506319, 0.95022164,
        0.95311615, 0.95570312]),
 'limiter': array([0., 0., 0., 0., 0., 2., 2., 2., 0., 0., 0., 0.]),
 'limval': array([0.51559692, 0.51559692, 0.51559692, 0.51559692, 0.51559692,
        2.39827465, 2.38265472, 2.40354223, 0.51559692, 0.51559692,
        0.51559692, 0.51559692])}

### Overall nutrient response factors

In [29]:
NutLimDiatOrig=np.where(DiatLimOrig['SiLim']<DiatLimOrig['NLim'],DiatLimOrig['SiLim'],DiatLimOrig['NLim'])
NutLimNanoOrig=NanoLimOrig['NLim']
NutLimDiatExp=np.where(DiatLimExp['SiLim']<DiatLimExp['NLim'],DiatLimExp['SiLim'],DiatLimExp['NLim'])
NutLimNanoExp=NanoLimExp['NLim']


### Now calculate diatom to flagellate overall growth ratios (multiply by max growth) when temperature and nutrient responses are considered in isolation:

###### * these rates would never occur in the model, even under repleat/optimal conditions, as the maximum of the nutrient and temperature responses are not equal to 1.

In [30]:
mumaxDiat=nml['zz_rate_r_diat']
mumaxNano=nml['zz_rate_r_nano']

## Get values to use in summary plots in Figure8_TempNutDependence.ipynb notebook

In [31]:
mumaxDiat/mumaxNano*NutLimDiatExp/NutLimNanoExp #'WY with CY Rivers Nutrient dependence only'

array([2.71312929, 2.70705032, 2.63109434, 2.25416726, 2.06368532,
       1.61223576, 1.44798715, 1.44531944, 2.39809586, 2.65546763,
       2.67459232, 2.69711616])

In [32]:
mumaxDiat/mumaxNano*TdepDiatExp/TdepNanoExp #'WY with CY Rivers Temperature dependence only'

array([2.86842105, 2.86842105, 2.86842105, 2.86842105, 2.45404626,
       1.72678252, 1.37297695, 1.4480068 , 1.91139694, 2.86842105,
       2.86842105, 2.86842105])

In [33]:
mumaxDiat/mumaxNano*TdepDiatExp/TdepNanoExp*NutLimDiatExp/NutLimNanoExp #'WY with CY Rivers both')


array([2.71312929, 2.70705032, 2.63109434, 2.25416726, 1.76556341,
       0.97056202, 0.69308269, 0.72961129, 1.59799172, 2.65546763,
       2.67459232, 2.69711616])